In [1]:
from ipynb.fs.full.data_extraction import init_spark,create_leagues,create_matches,read_file
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.sql.functions import corr,when,col

DataFrame[match_id: string, home_team_id: int, home_team: string, home_team_goal: int, league_id: int, B365H: double, B365D: double, B365A: double, away_team_id: int, away_team: string, away_team_goal: int, winner: string, country: string]


In [2]:
spark = init_spark()
sc = spark.sparkContext

In [3]:
match = read_file("Match.csv")
match_with_winner = match.withColumn("winner",when(match["home_team_goal"]<match["away_team_goal"],match["away_team_api_id"]).otherwise(
when(match["home_team_goal"]>match["away_team_goal"], match["home_team_api_id"]).otherwise("0")))

In [4]:
def string_indexer(dataframe,input_column,output_column):
    indexer = StringIndexer(inputCol = input_column,outputCol = output_column)
    return indexer.fit(dataframe).transform(dataframe)

In [5]:
correlation = {}
match_with_winner.cache()
for column, column_type in match_with_winner.dtypes:
    if column_type == "string":
        c = match_with_winner.select(corr(match_with_winner["winner"], match_with_winner[column]).alias("c")).collect()
        res = sc.parallelize(c).map(lambda x: x.c).collect()[0]
        correlation[column] = res


In [6]:
dict(filter(lambda x: x[1] is None,correlation.items()))

{'season': None,
 'goal': None,
 'shoton': None,
 'shotoff': None,
 'foulcommit': None,
 'card': None,
 'cross': None,
 'corner': None,
 'possession': None}

In [14]:
corners = read_file("corner_detail.csv")
# corners.select("corner").filter(corners["corner"].isNull() == False).show
print(corners.columns)
corners.select('match_id', 'n', 'player1', 'pos_x', 'pos_y').show()

['coordinates', 'corner', 'corners', 'del', 'elapsed', 'elapsed_plus', 'event_incident_typefk', 'id', 'match_id', 'n', 'player1', 'pos_x', 'pos_y', 'sortorder', 'spectators', 'stats', 'subtype', 'team', 'type', 'value']
+--------+---+-------+-----+-----+
|match_id|  n|player1|pos_x|pos_y|
+--------+---+-------+-----+-----+
|    1729|280|  38807| null| null|
|    1729|263|  40565| null| null|
|    1729|293|  38807| null| null|
|    1729|300|  24154| null| null|
|    1729|301|  38807| null| null|
|    1729|307|  24148| null| null|
|    1729|309|  24148| null| null|
|    1729|317|  40565| null| null|
|    1729|322|  38807| null| null|
|    1729|326|  24148| null| null|
|    1729|333|  24148| null| null|
|    1729|336|  30829| null| null|
|    1730|228|  39297| null| null|
|    1730|229|  39297| null| null|
|    1730|238|  30960| null| null|
|    1730|248|  39297| null| null|
|    1730|256|  39297| null| null|
|    1730|259|  23257| null| null|
|    1730|263|  23257| null| null|
|    1730|